<span style="font-size:36px"><b>Preprocess Language Model Corpus</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [61]:
import os
import re
import functools
import string
import random

import pandas as pd
import num2words
from tqdm import tqdm_notebook as tqdm

from gurih.models.utils import sparsity

# RegExp Collections

In [62]:
num_dict = {
    "0": "kosong",
    "1": "satu",
    "2": "dua",
    "3": "tiga",
    "4": "empat",
    "5": "lima",
    "6": "enam",
    "7": "tujuh",
    "8": "delapan",
    "9": "sembilan",
    "*": "",
    "x": "",
    " ": "",
    "-": ""
}

In [63]:
# 4,69% --> empat koma enam sembilan persen
repl_percentage_with_comma = lambda x: " ".join([num2words.num2words(int(num), lang='id') for num in x.group(1)]) + " koma " + " ".join([num2words.num2words(int(num), lang='id') for num in x.group(2)]) + " persen"
regex_percentage_with_comma = functools.partial(re.sub, "(\d+),(\d+)[\s]{0,1}%", repl_percentage_with_comma)

# rp400.000 --> 400.000
regex_currency = functools.partial(re.sub, "(rp)([\s]{0,1}\d)", r"\2")

# 400.000/bulan --> 400.000 per bulan
regex_per_month = functools.partial(re.sub, "(/)([\s]{0,2}bulan)", r" per \2")

# 3x24 jam --> 3 kali 24 jam
# 3x --> 3 kali 
# 3 x 24 jam --> 3 kali 24 jam
regex_frequency = functools.partial(re.sub, "(\d+)\s?x\s?(\d+)?", r"\1 kali \2")

# kontrak 380 170**xx15 96 --> kontrak tiga delapan kosong  satu tujuh kosong     satu lima  sembilan enam
regex_contract_number = functools.partial(re.sub, "(kontrak[\s]{0,2})([\d\s\*x]+)", lambda x: f"{x.group(1)}{' '.join([num_dict[num] for num in x.group(2)])}")

#  021-238432 --> kosong dua satu  dua tiga delapan empat tiga dua
regex_phone_number = functools.partial(re.sub, "(\s)(0[\d\s\*x-]+)", lambda x: f"{x.group(1)}{' '.join([num_dict[num] for num in x.group(2)])}")

# @hoewwedr --> ''
regex_remove_username = functools.partial(re.sub, "(@[A-Za-z0-9]+)", '')

# #yangkamumau --> ''
regex_remove_hashtag = functools.partial(re.sub, "(#[A-Za-z0-9]+)", '')

# https://api.whatsapp.com/send=?phone18273 --> ''
regex_remove_url = functools.partial(re.sub, "^https?:\/\/.*[\r\n]*", '')

# awdaw.....,,,, --> awdaw.,
regex_multiple_punctuations = functools.partial(re.sub, r'([.,])\1+', r'\1')

# gitu.aku sebel ,banget sama dia , dan kamu . --> gitu. aku sebel, banget sama dia, dan kamu. 
regex_triplet_punctuations = functools.partial(re.sub, r'(.)\s?(\.|,)\s?(.)?', r'\1\2 \3')

In [64]:
assert regex_percentage_with_comma("4,69%") == "empat koma enam sembilan persen"
assert regex_currency("rp400.000") == "400.000"
assert regex_per_month("400.000/bulan") == "400.000 per bulan"
assert regex_frequency("3x24 jam") == "3 kali 24 jam"
assert regex_frequency("3x") == "3 kali "
assert regex_frequency("3 x 24 jam") == "3 kali 24 jam"
assert regex_contract_number("kontrak 380 170**xx15 96") == "kontrak tiga delapan kosong  satu tujuh kosong     satu lima  sembilan enam"
assert regex_phone_number(" 021-238432") == " kosong dua satu  dua tiga delapan empat tiga dua"
assert regex_remove_username("@hoewwedr") == ''
assert regex_remove_hashtag("#yangkamumau") == ''
assert regex_remove_url("https://api.whatsapp.com/send=?phone18273") == ''
assert regex_multiple_punctuations("awdaw.....,,,,") == "awdaw.,"
assert regex_triplet_punctuations("gitu.aku sebel ,banget sama dia , dan kamu .") == "gitu. aku sebel, banget sama dia, dan kamu. "

# Instagram Comment Corpus

In [65]:
instagram_df = pd.read_csv("../../dataset/corpus/homecreditid_comments.csv")
instagram_df.shape

(26350, 4)

We only need those conversations in comment sections.

In [66]:
corpus = instagram_df['owner_comment'].values.tolist()

Run cell below to see what the comments looks like.

In [67]:
random.sample(corpus, 5)

['https://www.youtube.com/channel/UCvxef6NxP6D_1NBK8hBUAmQ https://youtu.be/ZZuwVpG0ICM https://youtu.be/-OcGPizXO8E subcribe ya',
 'Minn udah di umumin blm ini pemenang nya???',
 'Kolak, Biji Salak, Es Buah, Gorengan, Sop',
 'Ez first try😎',
 'Saya berhasil capture']

Save corpus statistics before cleaning.

In [68]:
before_cleaning_stats = sparsity(corpus)

## Cleaning

This processes must be done in order.

### Lower Casing

In [69]:
corpus = [s.lower() for s in corpus]

In [70]:
random.sample(corpus, 5)

['caranya gmana ka',
 '66 #tekatekuis',
 'min give away belum diumumin pemenangnya',
 'ada 12 kak',
 'susah ngajuiin blm 5 detik udah di tolak engga bgs']

### Remove @username

In [71]:
corpus = [regex_remove_username(s) for s in corpus]

In [72]:
random.sample(corpus, 5)

['23 easy',
 'hutang flexi fast 23 jt bayarnya sampe 43 jt... tolong bantuannya yg kasusnya sama, sedang kesusahan bayar.bunga jangan gede2...',
 'done 😘',
 'percayalah riba itu merugikan, kalau mau suatu barang tapi uang belum cukup mending tahan dulu aja tabung',
 'done, wish me luck']

### Remove Hashtag

In [73]:
corpus = [regex_remove_hashtag(s) for s in corpus]

In [74]:
random.sample(corpus, 5)

['_',
 '_rachmadi .moko  bangga yah alumni mentee kalian jadi artis sebentar lg ',
 'alhamdulillah dapet rumah baru',
 '  ',
 'ga ada kotak']

### Remove Url

In [75]:
corpus = [regex_remove_url(s) for s in corpus]

In [76]:
random.sample(corpus, 5)

['bismillah, menurut saya yang mengambil handphone korban adalah orang d karena dari gambar terlihat jelas bahwa orang tersebut mempunyai 2 smartphone dan yang harus di lihat ialah tampang muka si pria tersebut harus dicurigai,kenapa?karena dari gaya si pria tersebut seakan akan ingin mau menghindar atau mau menutup nutupi bahwasanya dia mencuri handphone korban _paranginangin    ',
 'ada 11',
 'ini home credit kl nelepon org kayak mau bangkrut yaaa... tlp lebih rajin drpd pacar.. yg ngutang siapa yg ditagih siapa.. mane mulutnya multiguna bgt lagi.. kalo telepon lgsg auto buka puasa kali yaaaaaaa...',
 'pengumuman pemenang giveaway 100k kapan min ',
 'kerusakan dapat diperhatikan sisi kiri bawa tas orang a.pencuri hp adalah orang c 5 orang a & c membawa benda sejenis pisau lipat.']

### Simplify Punctuations

1. `?`, `!` --> `.`
2. `;` --> `,`
3. `'` --> `<empty>`

In [77]:
simplified_punctuation_map = {"?": ".",
                              "!": ".",
                              ";": ",",
                              "'": ""}

In [78]:
corpus = [''.join([simplified_punctuation_map.get(c, c) for c in s]) for s in corpus]

In [79]:
random.sample(corpus, 5)

['ke kota makassar jg dong🙏',
 'saya dpt penwaran ny . smpt di tlp jg . tp gk tw klnjtnnya gmn min',
 'bener2 nggak waras karyawan kalian. umur berapa sih orang call center kalian. sampe ga tau waktu. ini maghrib woy. lagi buka puasa, lagi sibuk solat ngaji orang kalian telepon. fix mesti dilaporin sih ini, meresahkan masyarakat.',
 'nenekku lagi rehat addikku pakek gigi kawat baju putih nampak terlihat ternyata hantu yang telah lewat dieu :    .asra27  ',
 'min cek dm']

### Remove Non Alphanumerical + Punctuation Character

This is done according to the character map used in the model.

In [80]:
regex_remove_non_charmap = functools.partial(re.sub, '[^A-Za-z0-9., ]', '')

In [81]:
corpus = [regex_remove_non_charmap(s) for s in corpus]

In [82]:
random.sample(corpus, 5)

['gerimis, genangan, mendung, becek, dan badai',
 'mawar. jam . lampu. kancing. garam. lambang home credit',
 ' malam jumat nginap di rumah mertua. nonton dvd warkop setan kredit. butuh elektronik di tanggal tua, jangan ragu pergi ke home credit.     ayo ikutan giveaway homecreditid    ',
 'c karena di antaranya yang lain paling mencurigakan. tasnya diletakkan di depan sehingga lebih mudah menilep barang. pakai helm supaya kamera cctv tidak bisa menangkap mukanya dengan jelas. kalau yang lain pasti mikir2 mau mencuri karena mukanya dengan mudah tertangkap kamera cctv.  dia juga pakai sarung tangan yang bisa melindungi jarijarinya tidak meninggalkan jejak sidik jari agar polisi kesulitan menemukan identitasnya.  jaketnya juga menjadi tempat yang aman buat menyimpan laptop curian seandainya tasnya digeledah security satpam misalnya. pokoknya sempurna banget si c ini.  yuk ikutan  chudhori sweet ',
 'aku sebagai mahasiswa selalu membawa air minum dengan tumbler ke kampus. karena dengan it

### Percentage with Comma

In [83]:
corpus = [regex_percentage_with_comma(s) for s in corpus]

In [84]:
random.sample(corpus, 5)

['sudaaah... ayo menangkan.',
 'pantai ya pantai,ini lah hasil sreenshot  destinasi pilihan saya,ehmmm belum kesana aja udh bayangin ya seru seru ya ,enak kyk nya bisa berlibur ke pantai ,angin semilir sepoi sepoi,belari larian  kpn nech bisa kepantai .yukh ke pantai yukh sob      ',
 'min lihat dm saya.',
 '',
 'cari pesugihan ke gunung lawu pulangnya dikuntit sama hantu buruan beli yang kamu mau home credit siap membantu ariaabbas  alian  ']

### Currency

In [85]:
corpus = [regex_currency(s) for s in corpus]

In [86]:
random.sample(corpus, 5)

['aku udh ikutan yaa min  ',
 'so beautiful. tag to me ',
 'cek dm lah min',
 'pergi kebulan naik pitam bareng rombongan sirkus',
 'pencapaian saya di tahun 2019 ini,saya sebagai sales agent home credit sangat bangga karna bisa memberikan pelayan terbaik dan bisa mewujudkan barang impian nasabah terima kasih home credit.24  exo . ']

### Frequency

In [87]:
corpus = [regex_frequency(s) for s in corpus]

In [88]:
random.sample(corpus, 5)

['lets break the quizkotak pangkat 3 8 itu berarti kotak  2, karena 23  8. kemudian kotak x segitiga pangkat 3  54. berarti 2 kali segitiga pangkat 3  54, 54 dibagi 2  27 yg artinya segitiga pangkat 3  27 ini berarti segitiga  3 karena 3327. lalu kotak x segitiga x bulet x bulet 6, artinya 2 kali 3x bulet x bulet  6. ini berarti bulet  1 karena 2 kali 3x1 kali 1 6 . pertanyaannya  kotak  segitiga  bulet  berapa. artinya 2316 sehingga jawabannya 6 betul bukan. harus menang sih, karena gw udah jawab dan jabarkan pemecahan kuis ini. dan juga gamau kalah kece nih kalo pake hp xiaomi note 10 pro, biar bisa lebih produktif, berkarya, dan jadi membanggakan. yuk coba pecahin kuisnya    ',
 'bagus fotonyaa',
 'dapat capture pantai.. yeayy...saya akan berjemur dan menghirup udara pantai yg konon bisa jadi therapy buat orang yg punya penyakit asma sepertiku. lalu saya juga mau ajak suamiku untuk photo sesion alaala pra wedding..hehe maklum waktu nikah dulu gak sempet punya photo pra wed.  ngebaya

### Contract Number

In [89]:
corpus = [regex_contract_number(s) for s in corpus]

In [90]:
random.sample(corpus, 5)

['hai mimin, cek di ig aku ya.. semoga beruntung',
 'order skrng. pelangsing badan penggemuk badan pemutih badan pembersih jerawat peninggi badan dll',
 'bismillah . untuk perempuan  yang pertama jilbab, kedua celana dan untuk laki pertama baju a tidak memiliki gambar dibagian dada kiri sedangkan yang b memiliki gambar dibagian dada kirinya , kedua celana,ketiga sepatu alhamdulillah done',
 'yakin mau cicil disini jgn sampe lo nyesel akan di telp setiap hari,fuck',
 '']

### Phone Number

In [91]:
corpus = [regex_phone_number(s) for s in corpus]

In [92]:
random.sample(corpus, 5)

['belanja',
 'saya minta tolong mas atau mba homecredit,  jgn telpon no saya terus sehari bisa puluhan kali ditelpon, ini saya gk ngutang, tapi teman saya yang pake no saya sbg registrasi awal, tolong lah jgn ganggu..ini tiap hr diteror terus,  ini no tmn saya yg ngutang an ricky eka ferdian kosong delapan lima delapan dua satu enam tujuh kosong enam satu tujuh ',
 'sesat',
 'aku dapat gambar pantai. 3 pantai yg ingin aku datangi bersama teman2 ku  1. pantai tanjung bira bulukumba 2. pantai kuta bali 3. pantai pink lombok yang ingin saya lakukan di pantai tanjung bira sya mau sekali menyentuh pasir putihnya yg terkenal sehalus tepung, disana tuhh.. katanya bersih dan nyaman gak seramai kyak dipantai2 yg terkenal jdi bgus untuk menenangkan pikiran, kita jg bisa bermain snorkeling dan diving. trus di pantai kuta bali yg terkenal akan ombaknya yg bagus untuk berselancar dan menikmati sunset yg terkenal akan keindahan sunsetnya dan... pantai pink di lombok aku mauu jg ke sana pengen main2 

### Spell Number

In [93]:
regex_spell_number = functools.partial(re.sub, r'(\d+)', lambda x: ' ' + num2words.num2words(int(x.group(1)), lang='id') + ' ')

In [94]:
corpus = [regex_spell_number(s) for s in corpus]

In [95]:
random.sample(corpus, 5)

['min tolong bales dm ..',
 'promo hp bunga kosong dong min',
 'kota jakarta',
 'menurut saya,  yg berinsial nama nya d  orang nya. pencurinya  yang permainan nya rapi.  saat ia ingin beraksi dengan pencurian nyad ,ia ambil yg saat suasana si korban sia itu lagi tak terlena oleh si d pun beraksi melakukan nya.  dan setelah bus stop dan korban a telah sampai tujuan nya dan si a ingin menelepon temannya minta di jemput dan hp nya pun tak ada lagi, dan setelah akhir nya pun pencuri si d pun berhasil melakukan pencurian nya.',
 'gebyar promo cuci gudang   order  tiga  bal   satu  bal    ongkir se   eluang usaha kami tawarkan untuk anda  tersedia bal pakaian bekas import  tanpa cacatreject. daftar nama bal yang ready   jaket hoodie isi  tiga ratus  pcs harga  dua . nol . nol   jaket outdoor isi  dua ratus lima puluh  pcs harga  dua . lima ratus . nol   jaket jeans isi  tiga ratus  pcs harga  dua . lima ratus . nol   celana jeans panjang isi  dua ratus lima puluh  harga  dua . tujuh ratus . 

### Trim Multiple Spaces

In [96]:
corpus = [' '.join(s.split()) for s in corpus]

In [97]:
random.sample(corpus, 5)

['lima',
 'semangat',
 'lima',
 'lima min, matanya siwer ketutup an belek',
 'hilih mending jangan dah home credit mah pembohong wkwkw kapok dah enggak lagi bikin tambah puyeng dan kesel']

### Remove Multiple Punctuations

In [98]:
corpus = [regex_multiple_punctuations(s) for s in corpus]

In [99]:
random.sample(corpus, 5)

['kerenn',
 'maksudnya ini min .',
 'agak nyesel dulu pernah pake hc, karena databest kita dijual belikan. tiba dua banyak banget yang nawarin pinjaman ini itu. di tlp siang sore mlm, yang mau credit di hc atau selain hc mending ga usahlah. ganggu. kok kalian bisa sih nyebarin data best customer seenaknya atau di perjual beli kan. berharap suatu hari yang begini gini bisa di hukum',
 'jasa convert pulsa jadi saldo atm bank rate telkomsel tri xl axis seratus . nol up tsel delapan puluh tujuh three delapan puluh empat xl axis sembilan puluh empat tiga ratus . nol up tsel delapan puluh sembilan three delapan puluh enam xl axis sembilan puluh lima sembilan ratus sembilan puluh sembilan . nol up tsel sembilan puluh three delapan puluh tujuh xl axis sembilan puluh enam dua . empat ratus sembilan puluh sembilan . nol up tsel sembilan puluh dua three delapan puluh delapan xl axis sembilan puluh tujuh catatan sebelum transaksi silahkan baca terlebih dahulu syarat dan ketentuan transaksi. penjah

### (Optional) Triplet Punctuactions

If using subword model, this step is optional, since space is treated as a basic symbol.

In [100]:
corpus = [regex_triplet_punctuations(s) for s in corpus]

In [101]:
random.sample(corpus, 5)

['di medan untuk toko kamera yg di support dmana ya. ',
 'hasil capture saya adalah bioskop, tempat favorit saya dan keluarga karena disana kita bisa mencari hiburan dengan melihat film layar lebar. banyaknya film yang ditayangkan di bioskop seperti drama, komedi, horor, dsb sehingga kita bisa memiliki pilihan film yang ingin dilihat. bioskop sendiri memiliki manfaat sebagai penghibur serta penghilang stress bagi saya. belum seru rasanya kalo kesana tanpa kalian zaa',
 'pocong ganti piyama. genderuwo pun pake celana. jalan hidup kian bermakna. karena ada solusinya. .isme',
 'aku pilih orang e soalnya dia lagi jualin hp murah, siapa tau hasil curian dari orang yg di slide satu yuk ikutan navrat',
 'ini pantun horor dari saya, satu. ada peniti, ada kue pancong, hatihati, dibelakangmu ada pocong. dua. lalu aku buka mata, laraklirik kanankiri, tibatiba muncul seorang wanita, dengan wajah sangat ngeri. tiga. siangsiang kejepit pintu, suara tangis tak tertahan, tak usah takut kepada hantu, p

### Remove Empty String and Strip

In [102]:
corpus = [s.strip() for s in corpus if s != '']

In [103]:
random.sample(corpus, 5)

['baca dm nya',
 'jalanjalan ke jepang bersama keluarga sambil berfotofoto ootd dibawah pohon sakura ternyata pas dicetak ada yang ngga biasa ada mba sadako ikutan say cheese di dalam fotonya',
 'enam',
 'saya minta tolong mas atau mba homecredit, jgn telpon no saya terus sehari bisa puluhan kali ditelpon, ini saya gk ngutang, tapi teman saya yang pake no saya sbg registrasi awal, tolong lah jgn ganggu. ini tiap hr diteror terus, ini no tmn saya yg ngutang an ricky eka ferdian kosong delapan lima delapan dua satu enam tujuh kosong enam satu tujuh',
 'min cek dm saya']

### Remove One Word Only Documents

In [104]:
corpus = [s for s in corpus if len(s.split()) > 1]

In [105]:
random.sample(corpus, 5)

['satu. pulang kampung dua. angpau tiga. baju baru empat. tabungan lima. investasi',
 'ketika ada masalah barang hilang disuatu tempat pasti kita akan mencurigai dan bahkan menuduh orang dua yang ada disekitar kita, hal ini sangatlah wajar karena disamping panik dan emosi. . postingan diatas selain orang yang kehilangan laptop juga ada pria yang membawa box a, wanita yang selfie kemudian ada kue red velvet dan tas besar disampingnyab, pria berjaket sedang pesan menuc, dan wanita yang buru dua keluar sambil membawa tas kecil d. nah klo kita langsung menuduh orang dua tersebut diatas tanpa bukti yang jelas pasti nanti bakal terjadi keributan, dan ini biasanya yang diinginkan oleh pelaku pencurian, agar terhindar dari pengecekan. dari postingan diatas saya memilih b sebagai pencuri karena ada bukti yaitu tangan yang ada noda merah dan tisue yang jatuh bernoda merah dari kue dibawah meja pemilik laptop. kemungkinan wanita tersebut sedang makan kue dan memegang tissue, nah disaat pemilik la

## Compare Stats

Recalculate stats after cleaning.

In [106]:
def print_stats(stats):
    print(f"Number of documents: {stats['n']}")
    print(f"Sum of Ni: {stats['Ni']}")
    print(f"Vocab size: {len(stats['V'])}")
    print(f"Sparsity: {stats['Sd']}")

In [107]:
after_cleaning_stats = sparsity(corpus)

In [108]:
print_stats(before_cleaning_stats)

Number of documents: 26350
Sum of Ni: 550641
Vocab size: 82637
Sparsity: 0.9997471206402597


In [109]:
print_stats(after_cleaning_stats)

Number of documents: 22749
Sum of Ni: 512310
Vocab size: 42653
Sparsity: 0.9994720157837877


## Save Corpus and Vocab

In [110]:
with open("../../dataset/corpus/instagram_corpus.txt", 'w') as f:
    for document in corpus:
        f.write(f"{document}\n")

In [111]:
with open("../../dataset/corpus/instagram_vocab.txt", 'w') as f:
    for vocab in after_cleaning_stats['V']:
        f.write(f"{vocab}\n")

## (Optional) Sub Word Tokenizer

In case we want to use sub word token, or word pieces.

In [122]:
import sentencepiece as spm
spm.SentencePieceTrainer.train('--input=../../dataset/corpus/instagram_corpus.txt --model_prefix=m --vocab_size=8000')

True

In [123]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [124]:
print(sp.encode_as_pieces('homecredit kamu bisa'))
print(sp.encode_as_ids('homecredit kamu bisa'))

['▁homecredit', '▁kamu', '▁bisa']
[123, 72, 16]


In [127]:
for n in range(10):
    print(sp.sample_encode_as_pieces('homecredit kamu bisa', -1, 0.1))
for n in range(10):
    print(sp.sample_encode_as_ids('homecredit kamu bisa', -1, 0.1))

['▁', 'homecredit', '▁kamu', '▁bisa']
['▁ho', 'm', 'e', 'c', 'redit', '▁ka', 'mu', '▁bis', 'a']
['▁', 'homecredit', '▁kamu', '▁b', 'i', 's', 'a']
['▁', 'home', 'credit', '▁ka', 'mu', '▁', 'b', 'i', 's', 'a']
['▁', 'homecredit', '▁kam', 'u', '▁', 'bisa']
['▁', 'homecredit', '▁', 'ka', 'mu', '▁bi', 'sa']
['▁', 'h', 'o', 'm', 'e', 'c', 'r', 'e', 'd', 'it', '▁', 'k', 'a', 'mu', '▁b', 'i', 'sa']
['▁', 'homecredit', '▁ka', 'mu', '▁', 'b', 'isa']
['▁ho', 'me', 'c', 'redit', '▁k', 'am', 'u', '▁bisa']
['▁', 'home', 'c', 'r', 'e', 'd', 'i', 't', '▁', 'k', 'a', 'm', 'u', '▁bis', 'a']
[433, 2096, 106, 396, 5858, 11, 3504, 205, 16]
[11, 5089, 5101, 72, 16]
[433, 326, 1069, 5101, 131, 757, 205, 1031, 80]
[11, 5089, 5101, 11, 86, 80, 200, 205, 57, 42, 410]
[44, 396, 5858, 11, 86, 80, 219, 11, 2266]
[44, 5101, 72, 16]
[11, 94, 326, 1069, 5101, 4045, 205, 16]
[123, 72, 791, 410]
[1754, 200, 106, 5101, 72, 16]
[11, 5089, 5101, 72, 16]


In [128]:
# get 10 best
print(sp.nbest_encode_as_pieces('homecredit kamu bisa', 10))
print(sp.nbest_encode_as_ids('homecredit kamu bisa', 10))

[['▁homecredit', '▁kamu', '▁bisa'], ['▁', 'homecredit', '▁kamu', '▁bisa'], ['▁homecredit', '▁ka', 'mu', '▁bisa'], ['▁homecredit', '▁kamu', '▁', 'bisa'], ['▁homecredit', '▁kamu', '▁bis', 'a'], ['▁home', 'credit', '▁kamu', '▁bisa'], ['▁homecredit', '▁kamu', '▁b', 'isa'], ['▁homecredit', '▁kamu', '▁bi', 'sa'], ['▁homecredit', '▁kam', 'u', '▁bisa'], ['▁homecredit', '▁k', 'a', 'mu', '▁bisa']]
[[123, 72, 16], [11, 2390, 72, 16], [123, 380, 219, 16], [123, 72, 11, 2266], [123, 72, 1031, 80], [44, 5101, 72, 16], [123, 72, 57, 2752], [123, 72, 791, 410], [123, 4045, 205, 16], [123, 131, 80, 219, 16]]
